In [1]:
import os
import pickle
import numpy as np
from scipy.io import loadmat

import scfw.cov_estimation as ce
from scfw.frank_wolfe import frank_wolfe
from scfw.scopt import scopt

In [2]:
data_folder = '../data/'
problem_name = 'Lymph.mat'
results = {problem_name: {}}
results_folder = './results/'
results_file = os.path.join(results_folder, 'cov_estimation', problem_name + '.pckl')
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        try:
            results = pickle.load(f)
        except Exception:
            pass 

In [3]:
# load data
data = loadmat(os.path.join(data_folder, problem_name))
    
A = data['S']
p = A.shape[0]
X_0 = np.eye(p) / p

## Parameters initialization

In [4]:
#first set of parameters
Mf = 2
nu = 3
#running parameters

terminate_tol = 1e-10

FW_params={
    'iter_FW': 100,
    'line_search_tol': 1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

## Functions

In [5]:
func_x = lambda X: ce.cov_val(A, X)
func_beta = lambda X, S, beta, extra_param, extra_param_s: ce.cov_val(A, (1 - beta) * X + beta * S)
grad_x = lambda X, inv_X: ce.cov_grad(A, X, inv_X)
grad_beta = lambda X, S, beta, inv_X: ce.cov_grad(A, (1 - beta) * X + beta * S) # TODO: add inverse
hess_mult_x = lambda S, inv_X: ce.cov_hess_mult(S, inv_X)
hess_mult_vec_x = lambda S, trace_sum: ce.cov_hess_mult_vec(S, inv_X)
extra_func = lambda X: np.linalg.eigvalsh(X)
linear_oracle = lambda grad: ce.linear_oracle(grad)
prox_func = lambda X, L: ce.proj_map(X) #used for SCOPT

## Run FW

In [6]:
#run_alpha_policies = ["standard", "line_search", "icml", "backtracking"]
run_alpha_policies = ["standard"]

for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       X_0,
                       FW_params,
                       #hess=hess_x, 
                       #lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }

********* Algorithm starts *********
f: 3743.139569 Gap: 5.684342e-14
iter = 1, stepsize = 0.5, criterion = 0.49957392408111706, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 4143.640237 Gap: 5.870000e+02
iter = 2, stepsize = 0.4, criterion = 0.44713740270691854, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 4436.830895 Gap: 1.369667e+03
iter = 3, stepsize = 0.3333333333333333, criterion = 0.3726551396583188, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 4667.952041 Gap: 2.348000e+03
iter = 4, stepsize = 0.2857142857142857, criterion = 0.31586022213679094, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 4858.392221 Gap: 3.522000e+03
iter = 5, stepsize = 0.25, criterion = 0.2732874598042857, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 5020.038545 Gap: 4.891667e+03
iter = 6, stepsize = 0.2222222222222222, criterion = 0.2

KeyboardInterrupt: 

In [10]:
grad = grad_x(X_0, inv_X)

In [11]:
grad

array([[-5.86000000e+02,  1.98246239e-01,  2.27265618e-01, ...,
         1.01218126e-01,  1.81443299e-01, -4.68225435e-03],
       [ 1.98246239e-01, -5.86000000e+02,  8.49994237e-01, ...,
        -1.87059762e-01,  1.82687279e-01, -5.51225886e-02],
       [ 2.27265618e-01,  8.49994237e-01, -5.86000000e+02, ...,
        -2.60526925e-02,  1.77413834e-01, -1.70621545e-01],
       ...,
       [ 1.01218126e-01, -1.87059762e-01, -2.60526925e-02, ...,
        -5.86000000e+02, -2.05254916e-01, -8.54251376e-02],
       [ 1.81443299e-01,  1.82687279e-01,  1.77413834e-01, ...,
        -2.05254916e-01, -5.86000000e+02, -5.12942325e-02],
       [-4.68225435e-03, -5.51225886e-02, -1.70621545e-01, ...,
        -8.54251376e-02, -5.12942325e-02, -5.86000000e+02]])

In [12]:
S = linear_oracle(grad)

In [13]:
x_nxt = X_0 + 2/3 * (S - X_0)

In [16]:
np.sum(S)

-1.0

In [19]:
x_nxt

array([[0.00056786, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00056786, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00056786, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00056786, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00056786,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00056786]])

In [22]:
A = np.array([[1, -3], [-4, 3]])

In [13]:
x = X_0
f_vals = []
for i in range(100):
    f, inv_X = func_x(x)
    x = x - 0.01 * grad_x(x, inv_X)
    f_vals.append(f)

In [14]:
f_vals

[3743.1395692390934,
 2253.9285299320827,
 2101.179286305169,
 1948.5753972224338,
 1796.1892120512784,
 1644.1986204095851,
 1493.445091643542,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]